In [11]:
#Importing required libraries
import pandas as pd  #pandas for importing files and rearranging the data
import numpy as np
import seaborn as sb
from scipy import sparse #scipy for math functions
from sklearn.metrics.pairwise import cosine_similarity #sklearn for cosine similarity
import sys
import pickle
import matplotlib.pyplot as plt
import math
import seaborn as sns
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
dataset = pd.read_csv('data/dataset.csv')

if not os.path.isfile('train/train.csv'):
    # create the dataframe and store it in the disk for offline purposes..
    dataset.iloc[:int(dataset.shape[0]*0.80)].to_csv("train/train.csv", index=False)

if not os.path.isfile('test/test.csv'):
    # create the dataframe and store it in the disk for offline purposes..
    dataset.iloc[int(dataset.shape[0]*0.80):].to_csv("test/test.csv", index=False)

train = pd.read_csv("train/train.csv")
test = pd.read_csv("test/test.csv")

In [13]:
dataPD=train
dataPD.describe()

,userID,itemID,age,gender,category,crowdedness,creationDate,knowledgeOfemaleemaleSurroundings,timeAvailable,season,...,emotionalStability,temperature,budget,agreeableness,opennessToExperience,transport,companion,travelGoal,distance,rating
count,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,...,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000,11951.000000
mean,954.838842,39.090704,1.534767,0.735085,3.592335,0.230692,4.379885,0.343988,0.256046,0.348255,...,3.636432,0.559618,0.193206,3.432516,3.832148,0.079408,0.210359,0.238641,0.317463,3.490587
std,807.204692,46.340251,1.751779,0.700584,3.944764,0.602018,4.011983,0.877943,0.668133,0.951395,...,2.484113,1.326879,0.592201,2.335810,2.558459,0.476357,0.747389,1.266412,0.689645,1.298364
min,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,423.000000,6.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,909.000000,37.000000,0.000000,1.000000,2.000000,0.000000,4.000000,0.000000,0.000000,0.000000,...,4.000000,0.000000,0.000000,4.000000,5.000000,0.000000,0.000000,0.000000,0.000000,4.000000
75%,1431.000000,56.000000,3.000000,1.000000,4.000000,0.000000,9.000000,0.000000,0.000000,0.000000,...,6.000000,0.000000,0.000000,5.000000,6.000000,0.000000,0.000000,0.000000,0.000000,5.000000
max,11525.000000,249.000000,6.000000,2.000000,29.000000,3.000000,12.000000,3.000000,3.000000,4.000000,...,7.000000,6.000000,3.000000,7.000000,7.000000,4.000000,5.000000,9.000000,2.000000,5.000000


In [14]:
def showDataStats(inPD):
    """ Show selected statistics on given item ratings dataframe
    """
    # Get some numbers
    print("Rows: {0}".format(inPD.size))
    print("Distinct item: {0}".format(inPD['itemID'].unique().size))
    print("Distinct Users: {0}".format(inPD['userID'].unique().size))
  
    # Plot some histograms to get a feeling for the data
    numRatingsPerUserPD = inPD.groupby('userID')['itemID'].count()
    n, bins, patches = plt.hist(numRatingsPerUserPD.as_matrix(), 100, log=True, facecolor='blue', alpha=0.75)
    plt.xlabel('#Ratings')
    plt.ylabel('#Users')
    plt.title('#ratings per user')
    plt.grid(True)
    plt.show()

    numRatingsPerMoviePD = inPD.groupby('itemID')['userID'].count()
    n, bins, patches = plt.hist(numRatingsPerMoviePD.as_matrix(), 100, log=True, facecolor='blue', alpha=0.75)
    plt.xlabel('#Ratings')
    plt.ylabel('#item')
    plt.title('#ratings per item')
    plt.grid(True)
    plt.show()
showDataStats(dataPD)

Rows: 298425
Distinct item: 248
Distinct Users: 1620


AttributeError: 'Series' object has no attribute 'as_matrix'

In [21]:
filteredDataPD = dataPD

In [22]:
# Filter out the bottom 95% of the movies
numRatingsPerMoviePD = filteredDataPD.groupby('itemID')['userID'].count()
filterOutMoviesPD = numRatingsPerMoviePD[(numRatingsPerMoviePD <= np.percentile(numRatingsPerMoviePD, 95))]
filteredDataPD = filteredDataPD[~filteredDataPD['itemID'].isin(filterOutMoviesPD.index)]

In [23]:
distinctUsers = filteredDataPD['userID'].unique()
filterUsersPD = pd.DataFrame(distinctUsers, columns=['userID']).sample(frac=0.1)
filteredDataPD = filteredDataPD[filteredDataPD['userID'].isin(filterUsersPD['userID'])]

In [24]:
numRatingsPerUserPD = filteredDataPD.groupby('userID')['itemID'].count()
filterOutUsersPD = numRatingsPerUserPD[(numRatingsPerUserPD <= np.percentile(numRatingsPerUserPD, 10))
                              | (numRatingsPerUserPD >= np.percentile(numRatingsPerUserPD, 90))]
filteredDataPD = filteredDataPD[~filteredDataPD['userID'].isin(filterOutUsersPD.index)]

In [25]:
showDataStats(filteredDataPD)

Rows: 7498
Distinct item: 12
Distinct Users: 82


AttributeError: 'Series' object has no attribute 'as_matrix'